In [1]:
import tkinter as tk
from PIL import ImageTk, Image
import numpy as np
import colour as color

In [2]:
class Point:
    def __init__(self, x, y, z, color = "black"):
        self.x = x
        self.y = y
        self.z = z
        self.color = color

        self.matrix = np.array([x, y, z])

    def normalized(self):
        length = (self.x ** 2 + self.y ** 2 + self.z ** 2) ** 0.5
        return Point(self.x / length, self.y / length, self.z / length, self.color)

    def cross_product(self, point):
        return Point(self.y * point.z - self.z * point.y,
                    self.z * point.x - self.x * point.z,
                    self.x * point.y - self.y * point.x, 0)
    
    def dot_product(self, point):
        return self.x * point.x + self.y * point.y + self.z * point.z
    
    def __sub__(self, point):
        return Point(self.x - point.x, self.y - point.y, self.z - point.z, self.color)
    
    def light_vector(self, light):
        return (light - self).normalized()
    
    def observer_vector(self, camera):
        return (camera - self).normalized()
    
    def reflection_vector(self, light):
        return (light - self).normalized()

In [3]:
class Polygon:
    def __init__(self, v1, v2, v3):
        self.v1 = v1
        self.v2 = v2
        self.v3 = v3
        self.normal = self.calculate_normal()

    def calculate_normal(self):
        v1 = self.v2 - self.v1
        v2 = self.v3 - self.v1
        return v1.cross_product(v2).normalized()

In [4]:
class Camera:
    def __init__(self, u, v, w, origin, height, width, focus, far_plane, close_plane):
        self.u = u
        self.v = v
        self.w = w
        self.origin = origin
        self.height = height
        self.width = width
        self.focus = focus
        self.far_plane = far_plane
        self.close_plane = close_plane

        self.aspect_ratio = width / height

        self.camera_matrix = np.array([
            [self.u.x, self.v.x, self.w.x],
            [self.u.y, self.v.y, self.w.y],
            [self.u.z, self.v.z, self.w.z]
        ])

        self.transposed_camera_matrix = np.transpose(self.camera_matrix)

        self.origin_matrix = -np.matmul(self.transposed_camera_matrix, self.origin.matrix)

        self.transformation_matrix = np.array([
            [self.u.x, self.u.y, self.u.z, self.origin_matrix[0]],
            [self.v.x, self.v.y, self.v.z, self.origin_matrix[1]],
            [self.w.x, self.w.y, self.w.z, self.origin_matrix[2]],
            [0, 0, 0, 1]
        ])

        self.projection_matrix = np.array([
            [self.focus / self.width, 0, 0, 0],
            [0, self.focus / self.height, 0, 0],
            [0, 0, -(self.far_plane + self.close_plane) / (self.far_plane - self.close_plane), -2 * self.far_plane * self.close_plane / (self.far_plane - self.close_plane)],
            [0, 0, -1, 0]
        ]) 

In [5]:
class Screen:
    def __init__(self, width, height, window, camera, lights):
        self.width = width
        self.height = height
        self.window = window
        self.camera = camera
        self.ligts = lights

        # Calculates the delta between the center of the screen and the center of the window
        self.x_delta = (width - 1) / (camera.width * 2)
        self.y_delta = (height - 1) / (camera.height * 2)

        # Sets up the window
        self.window.geometry(str(width) + "x" + str(height))
        self.canvas = tk.Canvas(self.window, width = width, height = height)    
        self.canvas.configure(background = "black")
        self.canvas.pack()

        # Displays lights
        for light in self.ligts:
            self.draw_point(light, 5)

    # PRIVATE VARIABLES
    # Transforms a point in world space (x, y, z) to projection space (x, y, z, w)
    def _world_to_projection(self, point):
        point_4d = np.append(point.matrix, 1)
        transformed = np.matmul(self.camera.transformation_matrix, point_4d)
        projected = np.matmul(self.camera.projection_matrix, transformed)
        projected_scaled = projected / projected.max()
        return projected_scaled
    
    # Transforms a point in world space (x, y, z) to screen coordinates (x, y)
    def _world_to_screen_coordinate(self, point):
        point_2d = self._world_to_projection(point)
        display_x = (self.x_delta * point_2d[0]) + (self.x_delta * self.camera.width) + self.camera.width
        display_y = (self.y_delta * point_2d[1]) + (self.y_delta * self.camera.height) + self.camera.height
        return [round(display_x), round(display_y)]
    
    # Draws a point in world space (x, y, z) on the screen with the point's color
    def draw_point(self, point, size = 1):
        screen_coordinate = self._world_to_screen_coordinate(point)
        self.canvas.create_oval(screen_coordinate[0] - size, screen_coordinate[1] - size, screen_coordinate[0] + size, screen_coordinate[1] + size, fill = point.color, outline = point.color)

    # Draws a screen coordinate (x, y) on the screen with the given color
    def draw_coordinate(self, x, y, color = "white"):
        self.canvas.create_oval(x, y, x, y, fill = color, outline = color)

    # Creates a list of screen coordinates (x, y) between two points in world space (x, y, z)
    def calculate_line(self, a, b):
        # Converts points from world space to screen space
        point_a = self._world_to_screen_coordinate(a)
        point_b = self._world_to_screen_coordinate(b)

        # Calculates the difference between the points
        delta_x = point_b[0] - point_a[0]
        delta_y = point_b[1] - point_a[1]

        # Calculates the number of steps needed to draw the line on the larger axis
        if abs(delta_x) > abs(delta_y):
            s = abs(delta_x)
            delta_y = delta_y / s
            delta_x = delta_x / s
        else:
            s = abs(delta_y)
            delta_y = delta_y / s
            delta_x = delta_x / s

        # Interpolates color
        color_list = list(a.color.range_to(b.color, s + 1))

        # Draws the line
        display_coordinates_list = []
        for i in range(s + 1):
            display_coordinates_list.append((round(point_a[0] + i * delta_x), round(point_a[1] + i * delta_y)))

        return display_coordinates_list, color_list
    
    # Calculates the display coordinates of a line from two points in display space
    def calculate_line_display(self, a, b, a_color, b_color):
        # Calculates the difference between the points
        delta_x = b[0] - a[0]
        delta_y = b[1] - a[1]

        # Calculates the number of steps needed to draw the line on the larger axis
        if abs(delta_x) > abs(delta_y):
            s = abs(delta_x)
            delta_y = delta_y / s
            delta_x = delta_x / s
        else:
            s = abs(delta_y)
            delta_y = delta_y / s
            delta_x = delta_x / s

        # Interpolates color
        color_list = list(a_color.range_to(b_color, s + 1))

        # Draws the line
        display_coordinates_list = []
        for i in range(s + 1):
            display_coordinates_list.append((round(a[0] + i * delta_x), round(a[1] + i * delta_y)))

        return display_coordinates_list, color_list
    
    # Draws a line from two points in world space
    def draw_line(self, a, b):
        coordinates, color = self.calculate_line(a, b)
        for i in range(len(coordinates)):
            self.draw_coordinate(coordinates[i][0], coordinates[i][1], color[i])

    # Draws a triangle from three points in world space
    def draw_triangle(self, triangle):
        # Calculates the display coordinates of the vertex from where all lines will start
        line_beginning = self._world_to_screen_coordinate(triangle.v1)
        line_beginning_color = triangle.v1.color

        # Calculates the display coordinates of the vertices along v2 and v3 where the lines will end
        line_ends, line_ends_colors = self.calculate_line(triangle.v2, triangle.v3)

        # Draws the lines
        for i in range(len(line_ends)):
            points, colors = self.calculate_line_display(line_beginning, line_ends[i], line_beginning_color, line_ends_colors[i])

            # Draws the points
            for j in range(len(points)):
                self.draw_coordinate(points[j][0], points[j][1], colors[j])

    def calculate_vectors(self, point):
        light_vectors = []
        observer_vectors = []
        reflection_vectors = []

        for light in self.ligts:
            light_vectors.append(point.light_vector(light))
            observer_vectors.append(point.observer_vector(self.camera.origin))
            reflection_vectors.append(point.reflection_vector(light))

        return light_vectors, observer_vectors, reflection_vectors


In [6]:
def draw_cube(screen, vertices):
    screen.draw_line(vertices[4], vertices[0])
    screen.draw_line(vertices[4], vertices[5])
    screen.draw_line(vertices[0], vertices[1])
    screen.draw_line(vertices[5], vertices[1])

    screen.draw_line(vertices[4], vertices[6])
    screen.draw_line(vertices[0], vertices[2])
    screen.draw_line(vertices[5], vertices[7])
    screen.draw_line(vertices[1], vertices[3])

    screen.draw_line(vertices[6], vertices[7])
    screen.draw_line(vertices[6], vertices[2])
    screen.draw_line(vertices[2], vertices[3])
    screen.draw_line(vertices[7], vertices[3])

    screen.draw_line(vertices[4], vertices[1])
    screen.draw_line(vertices[5], vertices[3])
    screen.draw_line(vertices[7], vertices[2])
    screen.draw_line(vertices[6], vertices[0])
    screen.draw_line(vertices[4], vertices[7])
    screen.draw_line(vertices[1], vertices[2])


In [7]:
# Main window
window = tk.Tk()
window.title("Iluminación Phong")

''

In [8]:
# Camera declaration
u = Point(0, 0, 1)
v = Point(0, 1, 0)
w = Point(-1, 0, 0)
origin = Point(-4, 1, 0)
cam = Camera(u, v, w, origin, 1, 1, 1, 12, 3)

light_1 = Point(2, 2, 2, color.Color("#FFFFFF"))

# Light declaration
lights = [
    Point(2, 2, 2, color.Color("#FFFFFF")),
    Point(3, -3, 3, color.Color("#FFFFFF"))
]

# Scren declaration
screen = Screen(500, 500, window, cam, lights)

# Color declaration
red = color.Color("#FF0000")
green = color.Color("#00FF00")
blue = color.Color("#0000FF")
yellow = color.Color("#FFFF00")
cyan = color.Color("#00FFFF")
magenta = color.Color("#FF00FF")
orange = color.Color("#FF8000")
purple = color.Color("#8000FF")

# Vertices declaration
vertices = []
vertices.append(Point(1, 1, 1, red))
vertices.append(Point(1, 1, -1, green))
vertices.append(Point(1, -1, 1, blue))
vertices.append(Point(1, -1, -1, yellow))
vertices.append(Point(-1, 1, 1, cyan))
vertices.append(Point(-1, 1, -1, magenta))
vertices.append(Point(-1, -1, 1, orange))
vertices.append(Point(-1, -1, -1, purple))

point_a = Point(2, 2, 2, red)

draw_cube(screen, vertices)
#screen.color_lerp("#FF0000", "#00FF00", 10)

tri = Polygon(vertices[4], vertices[7], vertices[6])

screen.draw_triangle(tri)


In [9]:
# Tkinter loop
window.mainloop()